In [1]:
import webbrowser
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from PIL import ImageGrab, Image
import os.path
import math

In [2]:
# define path 
path = "C:\\Users\\m-bau\\Dokumente\\Studium\\Master\\3. Semester\\Data Science Project\\webscraping\\"

# Get list of streets in FFM

In [57]:
url = 'https://www.offenedaten.frankfurt.de/dataset/strassenverzeichnis-der-stadt-frankfurt-am-main/resource/be5982fe-ed79-42f4-acdc-57ca4737fb7a/download/strassenverzeichnis2022.csv'
url

'https://www.offenedaten.frankfurt.de/dataset/strassenverzeichnis-der-stadt-frankfurt-am-main/resource/be5982fe-ed79-42f4-acdc-57ca4737fb7a/download/strassenverzeichnis2022.csv'

In [73]:
streets = pd.read_csv(url, sep=";", encoding='latin1')
streets.shape

(4758, 15)

In [74]:
# remove duplicate street names
streets = streets.drop_duplicates(subset=['Straßenname', 'Postleitzahl'], keep='first')
streets.index = range(len(streets.index))
streets.shape

(3569, 15)

In [76]:
streets['Address'] = streets['Straßenname'] + ' ' + streets['Postleitzahl'].astype(str) + ' ' + 'Frankfurt'
streets['Straßenname'] = streets['Straßenname'].str.replace('*', '')
streets = streets.drop(['Folge', 'Hausnr. (von)', 'Hausnr. (bis)', 'Zusatz zur Hausnr (von)', 'Zusatz zur Hausnr (bis)'], axis=1)

In [77]:
# save data
streets.to_csv(path + "streets.csv", index= False)

# Get street information from OSM

In [78]:
streets = pd.read_csv(path + "streets.csv")

In [79]:
geolocator = Nominatim(user_agent="project", timeout=10)

In [80]:
location = geolocator.geocode(streets['Address'][3], timeout=3).raw

location

{'place_id': 102137570,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 8040949,
 'boundingbox': ['50.100842', '50.1011749', '8.6251804', '8.6254211'],
 'lat': '50.1010322',
 'lon': '8.6252843',
 'display_name': 'Ackermannstraße, Friedrich-Ebert-Siedlung, Gallus, Innenstadt 1, Frankfurt am Main, Hessen, 60326, Deutschland',
 'class': 'highway',
 'type': 'residential',
 'importance': 0.42000000000000004}

In [83]:
start_time = time.time()
i = 0
for a in streets['Address']:
    location = geolocator.geocode(a, timeout=3)
    if location is not None:
        streets.loc[i, 'Latitude'] = location.latitude
        streets.loc[i, 'Longitude'] = location.longitude
        streets.loc[i, 'class'] = location.raw['class']
        streets.loc[i, 'type'] = location.raw['osm_type']
    else:
        next
    i = i + 1
print("--- %s seconds ---" % (time.time() - start_time))

--- 1786.276083946228 seconds ---


In [85]:
streets.tail()

,Straßennummer,Straßenname,Ortsbezirk,Stadtteil Name,Stadtbezirk,Stadtbezirksvorst,Schiedsleutebezirk,Sozialrathaus Name,Polizeirevier,Postleitzahl,Address,Latitude,Longitude,class,type
3564,2735,Zur Untermühle,14,Harheim,660,14.81,14,Nord,14.0,60437,Zur Untermühle 60437 Frankfurt,50.184872,8.692604,highway,way
3565,2577,Zur Waldau,6,Schwanheim,532,6.37,6A,Sachsenhausen,10.0,60529,Zur Waldau 60529 Frankfurt,50.083565,8.610929,highway,way
3566,3201,Züricher Straße,15,Nieder-Eschbach,670,15.82,15,Nord,14.0,60437,Züricher Straße 60437 Frankfurt,50.196453,8.652494,highway,way
3567,2578,Zwingergasse,1,Innenstadt,50,1.02,1,Ost,1.0,60313,Zwingergasse 60313 Frankfurt,50.116081,8.676417,highway,way
3568,2580,Zwischenstraße,5,Sachsenhausen-N.,331,5.31,5A,Sachsenhausen,8.0,60594,Zwischenstraße 60594 Frankfurt,50.103039,8.694130,highway,way


In [25]:
#streets = streets.sort_values('Address')
streets.index = range(len(streets.index))
streets.head()

,Address,Latitude,Longitude,Straßennummer,Straßenname,Folge,Hausnr. (von),Zusatz zur Hausnr (von),Hausnr. (bis),Zusatz zur Hausnr (bis),Ortsbezirk,Stadtteil Name,Stadtbezirk,Stadtbezirksvorst,Schiedsleutebezirk,Sozialrathaus Name,Polizeirevier,Postleitzahl,class,type
0,Abflugring (Flughafen Ffm.) 60549 Frankfurt,50.052218,8.578938,3083,Abflugring (Flughafen Ffm.),NaN,NaN,NaN,NaN,NaN,5,Flughafen,327,5.34,5A,Sachsenhausen,19.0,60549,highway,way
1,Abtsgäßchen 60594 Frankfurt,50.104464,8.688812,1,Abtsgäßchen,NaN,NaN,NaN,NaN,NaN,5,Sachsenhausen-N.,300,5.31,5A,Sachsenhausen,8.0,60594,NaN,NaN
2,Achenbachstraße 60596 Frankfurt,50.097490,8.676911,2,Achenbachstraße,NaN,NaN,NaN,NaN,NaN,5,Sachsenhausen-N.,322,5.33,5A,Sachsenhausen,8.0,60596,NaN,NaN
3,Ackermannstraße 60326 Frankfurt,50.101032,8.625284,3,Ackermannstraße,NaN,NaN,NaN,NaN,NaN,1,Gallus,154,1.05,1,Gallus,16.0,60326,NaN,NaN
4,Adalbert-Stifter-Straße 60431 Frankfurt,50.145032,8.663405,4,Adalbert-Stifter-Straße,ungerade,NaN,NaN,NaN,NaN,9,Ginnheim,441,9.67,9,Nord,12.0,60431,NaN,NaN


In [86]:
# save data
streets.to_csv(path + "ffm.csv", index= False)

# Create subset for survey

In [87]:
streets = pd.read_csv(path + "ffm.csv")

In [3]:
streets = pd.read_csv(path + "ffm.csv")
# create subset 

np.random.seed(42)
streets = streets.sample(frac=1)

# number of survey streets
n_survey = 500

survey_streets = pd.DataFrame.sample(streets, n=n_survey)
survey_streets['Ortsbezirk'].unique()
survey_streets.index = range(len(survey_streets.index))

In [4]:
survey_streets.shape

(500, 15)

In [90]:
survey_streets.head()

,Straßennummer,Straßenname,Ortsbezirk,Stadtteil Name,Stadtbezirk,Stadtbezirksvorst,Schiedsleutebezirk,Sozialrathaus Name,Polizeirevier,Postleitzahl,Address,Latitude,Longitude,class,type
0,929,Germaniaplatz,4,Bornheim,240,4.23,4,Ost,6.0,60385,Germaniaplatz 60385 Frankfurt,50.125200,8.701290,highway,way
1,1093,Hattsteiner Straße,7,Rödelheim,401,7.55,7,Bockenheim,11.0,60489,Hattsteiner Straße 60489 Frankfurt,50.129838,8.615823,highway,way
2,642,Dietesheimer Straße,11,Fechenheim,520,11.78,11,Ost,18.0,60386,Dietesheimer Straße 60386 Frankfurt,50.116658,8.773124,highway,way
3,2700,Schwalbenweg,12,Kalbach-Riedberg,650,12.79,12,Nord,14.0,60437,Schwalbenweg 60437 Frankfurt,50.187611,8.631370,highway,way
4,2403,Usinger Straße,4,Bornheim,271,4.23,4,Ost,6.0,60389,Usinger Straße 60389 Frankfurt,50.130396,8.706119,highway,way


## Take screenshots from Google Streetview

In [11]:
# Scrape pics from Street view
for i in survey_streets.index:
    file = path + "pics\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg'
    if os.path.isfile(file) is True:
        next
    else:
        # define URL 
        url = 'http://maps.google.com/maps?q=&layer=c&cbll=' + survey_streets.loc[i]['Latitude'].astype(str) + ',' + survey_streets.loc[i]['Longitude'].astype(str) + '&cbp=11,0,0,0,0' 

        # open webbrowser which redirects you to different URL (takes some seconds)
        webbrowser.open(url)
        time.sleep(10)

        # take screenshot and save it
        snapshot = ImageGrab.grab()
        save_path = path + "pics\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg'
        snapshot.save(save_path)


In [12]:
# Crop screenshots
for i in survey_streets.index:
    img = Image.open(path + "\\pics\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg')
    if len(img.fp.read()) > 180000:
        area = (300, # rigth
                350, # upper
                1600, # left
                1100) # lower
        cropped_img = img.crop(area)
        #cropped_img.show() 
        save_path = path +"survey_pics\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg'
        cropped_img.save(save_path)
    else:
            next

484

In [6]:
survey_streets.head()

,Straßennummer,Straßenname,Ortsbezirk,Stadtteil Name,Stadtbezirk,Stadtbezirksvorst,Schiedsleutebezirk,Sozialrathaus Name,Polizeirevier,Postleitzahl,Address,Latitude,Longitude,class,type
0,929,Germaniaplatz,4,Bornheim,240,4.23,4,Ost,6.0,60385,Germaniaplatz 60385 Frankfurt,50.125200,8.701290,highway,way
1,1093,Hattsteiner Straße,7,Rödelheim,401,7.55,7,Bockenheim,11.0,60489,Hattsteiner Straße 60489 Frankfurt,50.129838,8.615823,highway,way
2,642,Dietesheimer Straße,11,Fechenheim,520,11.78,11,Ost,18.0,60386,Dietesheimer Straße 60386 Frankfurt,50.116658,8.773124,highway,way
3,2700,Schwalbenweg,12,Kalbach-Riedberg,650,12.79,12,Nord,14.0,60437,Schwalbenweg 60437 Frankfurt,50.187611,8.631370,highway,way
4,2403,Usinger Straße,4,Bornheim,271,4.23,4,Ost,6.0,60389,Usinger Straße 60389 Frankfurt,50.130396,8.706119,highway,way


We manually droped blurred pictures, street without streetview or non-streetview pictures

In [19]:
survey_streets['survey'] = 0

for i in survey_streets.index[:100]:
    file = path + "survey_pics\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg'
    if os.path.isfile(file) is True:
        os.path.isfile(file)
        survey_streets.loc[i,'survey'] = 1

survey_streets = survey_streets.loc[survey_streets['survey'] == 1]

from the 300 survey street we got 189 pictures whichbcan be used for the survey

- split pics in training and test data: 75% - 25% ??
- split training data into 80% traing set and 20% validation set
- size/ number of obs. of training set??


In [21]:
train_data = survey_streets.sample(frac=0.75, random_state=25)
test_data = survey_streets.drop(train_data.index)

In [22]:
survey_streets['subsample'] = "training"
for i in survey_streets.index:
    if i in test_data.index:
        survey_streets.loc[i, 'subsample'] = "test"
        img = Image.open(path + "survey_pics\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg')
        img.save(path + "test\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg')
    else:
        img = Image.open(path + "survey_pics\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg')
        img.save(path + "training\\" + survey_streets.loc[i]['Straßenname'] + '_' + survey_streets.loc[i]['Postleitzahl'].astype(str) + '.jpeg')
